# Request School Location Data from GigaMaps APIs

![Giga](https://www.geneve-int.ch/sites/default/files/styles/scale_450x450/public/2024-07/Logo-GIGA.jpg)

In [ ]:
# Install libraries as needed

import requests
import pandas as pd
import geopandas as gpd
from IPython.display import display, clear_output

In [ ]:
# Assign the the ISO code for the country you would like school location data for as a string to country_code variable.
# The list of countries available on GigaMaps (with ISO codes) can be downloaded using the GigaMaps Country API. https://maps.giga.global/docs/explore-api

country_code = "BRA"

In [ ]:
# Assign your unique API key as a string to variable api_key (if you don't have an API key, you can request one from Giga Maps)
api_key = "9grGJQwOziA1fJzBd7RVHiZGERFJbvDBnteo4S4yQYrzsi23j1FjYPVlj2T1#HZU#_ymdRIUAjNys0Z#fF86pTZjW07vIVBbJ#O2p6SC_XPRn#1VJy6YchOTyHpYkLCR8mow*4W*J6kXR_Cz12WO_lpzItRhNUpBUC7zPrfxUiBfU8J05f2DBpRMQ0Gn77IowoKlGqymfRBW9ZsEg_hW3OFsqcmi7aK#a86wRLwt1kuUTZjljf8*ctp6Decnx1n0zvW*j*DP"

In [ ]:
# URL for School Location API endpoint (using selected country)
BASE_URL = f"https://uni-ooi-giga-maps-service.azurewebsites.net/api/v1/schools_location/country/{country_code}"

#Headers
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

In [ ]:
# Parameters (see Giga Maps API docs)
# Requests need to be paginated- split up into multiple requests to prevent overloading the API
page = 1

# Current max page size is 1000, meaning each request sends 
size = 1000

In [ ]:
# Creates a list to store all the responses from API requests
all_schools = []

# Loop through pages until all school location data has been collected
while True:
    params = {
        "page": page,
        "size": size
    }
    
   # Request schools from the API endpoint 
    response = requests.get(BASE_URL, headers=headers, params=params)

    # Ensure that the response from the API is correct. With an incorrect response, no more requests will be sent
    if response.status_code != 200:
        print(f"Request failed at page {page}: {response.status_code}")
        break

    # Extract the schools returned from the API
    data = response.json()['data']

    # Check that the most recent request receives data in the response
    if not data or len(data) == 0:
        print(f"No more data returned at page {page}.")
        break

    #Continuously add newly fetched schools to the all_schools list
    all_schools.extend(data)
    clear_output(wait=True)
    display(f"Fetched page {page}, items received: {len(data)}")
    page += 1

In [ ]:
#Convert to pandas DataFrame
school_data = pd.DataFrame(all_schools)

In [ ]:
school_data.describe()

In [ ]:
# Create geopandas DataFrame
school_data = gpd.GeoDataFrame(school_data, geometry=gpd.points_from_xy(school_data.longitude, school_data.latitude), crs="4326")

In [ ]:
# Save the filtered data to a GeoJSON file
school_data.to_file("school_data.geojson", driver="GeoJSON")